In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, Float, String
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
import math
output_notebook()

# SQLAlchemy setup
Base = declarative_base()

In [ ]:
class TrainingData(Base):
    __tablename__ = 'training_data'
    id = Column(Integer, primary_key=True)
    x_value = Column(Float)
    y1 = Column(Float)
    y2 = Column(Float)
    y3 = Column(Float)
    y4 = Column(Float)

class IdealFunction(Base):
    __tablename__ = 'ideal_functions'
    id = Column(Integer, primary_key=True)
    x_value = Column(Float)
    y1 = Column(Float)
    y2 = Column(Float)
    y3 = Column(Float)
    y4 = Column(Float)  # Extend to y50 as needed

class TestResult(Base):
    __tablename__ = 'test_results'
    id = Column(Integer, primary_key=True)
    x_value = Column(Float)
    y_value = Column(Float)
    delta_y = Column(Float)
    ideal_function_number = Column(Integer)


In [ ]:
from pathlib import Path

db_file = Path("function_matching.db")
if db_file.exists():
    db_file.unlink()
    print("Corrupted database deleted.")
else:
    print("Database file not found.")


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

Base = declarative_base()

engine = create_engine('sqlite:///function_matching.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()
print("✅ New database created successfully.")


In [ ]:
#loading the  CSV Files
train_df = pd.read_csv('train.csv')
ideal_df = pd.read_csv('ideal.csv')
test_df = pd.read_csv('test.csv')


In [ ]:
def least_square_deviation(y_true, y_pred):
    return np.sum((y_true - y_pred) ** 2)

def find_best_ideal_matches(train_df, ideal_df):
    best_matches = {}
    for y_col in ['Y1', 'Y2', 'Y3', 'Y4']:
        min_dev = float('inf')
        best_func = None
        for ideal_col in ideal_df.columns[1:]:  # skip X
            dev = least_square_deviation(train_df[y_col], ideal_df[ideal_col])
            if dev < min_dev:
                min_dev = dev
                best_func = ideal_col
        best_matches[y_col] = best_func
    return best_matches


In [ ]:
def map_test_data(test_df, ideal_df, best_matches):
    results = []
    for _, row in test_df.iterrows():
        x_val, y_val = row['X'], row['Y']
        matched = False
        for idx, (train_col, ideal_col) in enumerate(best_matches.items(), 1):
            ideal_row = ideal_df[ideal_df['X'] == x_val]
            if not ideal_row.empty:
                ideal_y = ideal_row[ideal_col].values[0]
                delta = abs(y_val - ideal_y)
                max_dev = np.max(np.abs(train_df[train_col] - ideal_df[ideal_col]))
                if delta <= max_dev * sqrt(2):
                    results.append({
                        'x_value': x_val,
                        'y_value': y_val,
                        'delta_y': delta,
                        'ideal_function_number': idx
                    })
                    matched = True
                    break
        if not matched:
            results.append({
                'x_value': x_val,
                'y_value': y_val,
                'delta_y': None,
                'ideal_function_number': None
            })
    return results


In [ ]:
def save_results(results):
    for r in results:
        session.add(TestResult(**r))
    session.commit()

def plot_results(train_df, ideal_df, results, best_matches):
    p = figure(title="Function Matching", x_axis_label='X', y_axis_label='Y')
    p.scatter(train_df['X'], train_df['Y1'], color='blue', size=6, legend_label='Training Y1')

    ideal_col = best_matches['Y1']
    p.line(ideal_df['X'], ideal_df[ideal_col], color='green', line_width=2, legend_label=f'Ideal {ideal_col}')

    import pandas as pd
    results_df = pd.DataFrame(results)
    mapped = results_df[results_df['ideal_function_number'] == 1]
    p.square(mapped['x_value'], mapped['y_value'], color='red', size=6, legend_label='Test Mapped')

    p.legend.location = 'top_left'
    show(p)


In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

output_notebook()

# Sample data
data = {'x': [1, 2, 3, 4, 5], 'y': [6, 7, 2, 4, 5]}
source = ColumnDataSource(data)

# Create plot
p = figure(title="Updated Bokeh Plot", x_axis_label='X', y_axis_label='Y')
p.scatter(x='x', y='y', source=source, size=10, marker='circle', color="navy", alpha=0.6)

show(p)
